In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import tensorflow_probability as tfp
import numpy as np
import matplotlib.pyplot as plt
import corner
import importlib
import logging
import os
from functions import load_test_data, chi2_graph

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
importlib.reload(logging)
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(gpus[1],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=20000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
    print(e)

2 Physical GPUs, 1 Logical GPU


In [2]:
#3flavor_poisson, nsi_poisson
learn_target = 'nsi_poisson'
data_index = 1

data_all = load_test_data(learn_target, data_index)

In [ ]:
gaussian_number = 10
latent_dim = 10

In [ ]:
model_name = '/gaussian_number/10/'
path = './cvae/' + learn_target +  model_name
encoder1 = load_model(path + "encoder_1.h5", compile=False)
encoder2 = load_model(path + "encoder_2.h5", compile=False)
decoder = load_model(path + "decoder.h5", compile=False)

In [ ]:
if learn_target == '3flavor_poisson':
    data_poisson_array = data_all[:1]/1000

    mean, log_var, weight = encoder2.predict(data_poisson_array)

    mean = tf.reshape(mean, (-1, gaussian_number, latent_dim))
    log_var = tf.reshape(log_var, (-1, gaussian_number, latent_dim))
    weight = tf.reshape(weight, (-1, gaussian_number))

    SMALL_CONSTANT = 1e-12
    temp_var = SMALL_CONSTANT + tf.exp(log_var)
    test_sampling = tfp.distributions.MixtureSameFamily(
                mixture_distribution=tfp.distributions.Categorical(logits=weight),
                components_distribution=tfp.distributions.MultivariateNormalDiag(
                loc=mean,
                scale_diag=tf.sqrt(temp_var)))

    sample_amount= 100000
    sample_times = 10
    prediction = []
    for i in range(sample_times):
        Z3 = np.reshape(test_sampling.sample(sample_shape=(sample_amount)), (sample_amount, latent_dim))
        reconstruction_mean, reconstruction_var = decoder.predict([Z3, np.array(([data_poisson_array[0]]*sample_amount))])
        temp_var = SMALL_CONSTANT + tf.exp(reconstruction_var)
        reconstruction_parameter = tfp.distributions.MultivariateNormalDiag(
                                loc=reconstruction_mean,
                                scale_diag=temp_var)
        prediction.extend(reconstruction_parameter.sample().numpy())

    prediction_NO = np.array(prediction)
    data_poisson_array = data_all[1:2]/1000

    mean, log_var, weight = encoder2.predict(data_poisson_array)

    mean = tf.reshape(mean, (-1, gaussian_number, latent_dim))
    log_var = tf.reshape(log_var, (-1, gaussian_number, latent_dim))
    weight = tf.reshape(weight, (-1, gaussian_number))

    SMALL_CONSTANT = 1e-12
    temp_var = SMALL_CONSTANT + tf.exp(log_var)
    test_sampling = tfp.distributions.MixtureSameFamily(
                mixture_distribution=tfp.distributions.Categorical(logits=weight),
                components_distribution=tfp.distributions.MultivariateNormalDiag(
                loc=mean,
                scale_diag=tf.sqrt(temp_var)))

    sample_amount= 100000
    sample_times = 10
    prediction = []
    for i in range(sample_times):
        Z3 = np.reshape(test_sampling.sample(sample_shape=(sample_amount)), (sample_amount, latent_dim))
        reconstruction_mean, reconstruction_var = decoder.predict([Z3, np.array(([data_poisson_array[0]]*sample_amount))])
        temp_var = SMALL_CONSTANT + tf.exp(reconstruction_var)
        reconstruction_parameter = tfp.distributions.MultivariateNormalDiag(
                                loc=reconstruction_mean,
                                scale_diag=temp_var)
        prediction.extend(reconstruction_parameter.sample().numpy())

    prediction_IO = np.array(prediction)
    np.savez('./IO_contour/' + learn_target + '/cvae.npz', prediction_IO)
    np.savez('./NO_contour/' + learn_target + '/cvae.npz', prediction_NO)

In [ ]:
if learn_target == 'nsi_poisson':
    data_poisson_array = data_all[:1]/1000

    mean, log_var, weight = encoder2.predict(data_poisson_array)

    mean = tf.reshape(mean, (-1, gaussian_number, latent_dim))
    log_var = tf.reshape(log_var, (-1, gaussian_number, latent_dim))
    weight = tf.reshape(weight, (-1, gaussian_number))

    SMALL_CONSTANT = 1e-12
    temp_var = SMALL_CONSTANT + tf.exp(log_var)
    test_sampling = tfp.distributions.MixtureSameFamily(
                mixture_distribution=tfp.distributions.Categorical(logits=weight),
                components_distribution=tfp.distributions.MultivariateNormalDiag(
                loc=mean,
                scale_diag=tf.sqrt(temp_var)))

    sample_amount= 100000
    sample_times = 10
    prediction = []
    for i in range(sample_times):
        Z3 = np.reshape(test_sampling.sample(sample_shape=(sample_amount)), (sample_amount, latent_dim))
        reconstruction_mean, reconstruction_var = decoder.predict([Z3, np.array(([data_poisson_array[0]]*sample_amount))])
        temp_var = SMALL_CONSTANT + tf.exp(reconstruction_var)
        reconstruction_parameter = tfp.distributions.MultivariateNormalDiag(
                                loc=reconstruction_mean,
                                scale_diag=temp_var)
        prediction.extend(reconstruction_parameter.sample().numpy())

    prediction_NO = np.array(prediction)
    np.savez('contour/' + learn_target +  '_{}/cvae.npz'.format(data_index), prediction)